In [34]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns
import os
import csv
import xgboost as xgb
from sklearn.linear_model import Ridge
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor

a = int

In [2]:
os.chdir('D:\\test\project')

In [3]:
df = pd.read_csv('shipping_data_new.csv', sep=',')

In [4]:
df

,name,price ($),weight (kg),length (m),width (m),height (m),shipment date,destination port,volume,country,category
0,Camera Bag,37.66,1.10,0.40,0.39,0.26,2023-03-19,Port of Singapore (Singapore),0.040560,Singapore,Accessories
1,Portable Bluetooth Keyboard,144.65,0.39,0.11,0.06,0.03,2023-03-21,Port of Busan (South Korea),0.000198,Korea,Electronics
2,Large Flat Rate Box,38.57,0.97,0.79,0.55,0.35,2023-03-25,Port of Tianjin (China),0.152075,China,Stationery
3,Ceramic Tiles,10.34,6.22,0.36,0.37,0.02,2023-03-15,Port of Shanghai (China),0.002664,China,Materials
4,Garden Hose,21.63,1.18,17.77,0.27,0.13,2023-03-25,Port of Tianjin (China),0.623727,China,Garden
...,...,...,...,...,...,...,...,...,...,...,...
263816,Canister,23.58,3.80,0.38,0.37,0.28,2023-03-13,Port of Busan (South Korea),0.039368,Korea,Materials
263817,Plant,15.98,1.20,0.42,0.29,0.34,2023-03-15,Port of Tokyo (Japan),0.041412,Japan,Garden
263818,Pallete of Dog Food,223.05,477.58,1.00,0.92,1.48,2023-03-19,Port of Tokyo (Japan),1.361600,Japan,More food
263819,Coffee Maker,169.09,2.52,0.26,0.20,0.29,2023-03-20,Port of Shanghai (China),0.015080,China,Household Appliances


In [31]:
features = df[['category', 'weight (kg)', 'width (m)', 'length (m)', 'height (m)']]
target = df['price ($)']
features = pd.get_dummies(features)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

model = RandomForestRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 1093964745.8941636


In [32]:
## Рассмотрим модель линейной регрессии
for a in np.arange(0, 1.1, 0.1):
    features = df[['category', 'weight (kg)', 'width (m)', 'length (m)', 'height (m)']]
    target = df['price ($)']

    features = pd.get_dummies(features)
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    model = Ridge(alpha=a)  
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)


    mse = mean_squared_error(y_test, y_pred)
    print("Mean Squared Error:", mse)

Mean Squared Error: 3420138175.8845024
Mean Squared Error: 3420141988.3449197
Mean Squared Error: 3420142947.5679207
Mean Squared Error: 3420143907.05592
Mean Squared Error: 3420144866.8088727
Mean Squared Error: 3420145826.8267345
Mean Squared Error: 3420146787.109465
Mean Squared Error: 3420147747.657019
Mean Squared Error: 3420148708.469354
Mean Squared Error: 3420149669.5464263
Mean Squared Error: 3420150630.8881917


In [ ]:
##Как мы видим, данные не чувствительны к регуляризации и значения ошибки практически не изменяется с изменением alpha

In [10]:
##Попробуем построить модель случайного леса с использованием сетки гиперпараметров для поиска.
features = df[['category', 'weight (kg)', 'width (m)', 'length (m)', 'height (m)']]
target = df['price ($)']

features = pd.get_dummies(features)

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

model = RandomForestRegressor()

param_grid = {'n_estimators': [100, 200], 'max_depth': [None, 5], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'max_features': ['sqrt']  }
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Лучшие гиперпараметры:", best_params)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Лучшие гиперпараметры: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Mean Squared Error: 1027277771.5540844


In [47]:
features = df[['category', 'weight (kg)', 'width (m)', 'length (m)', 'height (m)']]
target = df['price ($)']
features = pd.get_dummies(features)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
model = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, reg_lambda=1, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 1093167688.4910364


In [ ]:
## Таким образом, видим, что наименьшая ошибка была получена при построении модели случайного леса, 